In [4]:
versioninfo()

Julia Version 0.6.0
Commit 903644385b* (2017-06-19 13:05 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-4700MQ CPU @ 2.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


In [2]:
using DataFrames, Distributions, GLM, MixedModels, RCall

In [3]:
p  = 10
n = 100
js=collect(Iterators.product('A':'A'+p-1, 1:n))[:]
df = DataFrame(gene=[js[i][1] for i = 1:size(js)[1]], cell=[js[i][2] for i = 1:size(js)[1]])
gene_effect = rand(Normal(0, 2), p)
cell_effect = rand(Normal(0,1), n)
df[:counts] = cell_effect[df[:cell]]+gene_effect[convert(Array{Int64,1},df[:gene])-64]+rand(Normal(2, 0.1), n*p) 
df[:counts] = [rand(Poisson(lambda)) for lambda in exp(df[:counts])] 
head(df)

Nullable{T}) in module Base at nullable.jl:238 overwritten in module NullableArrays at C:\Users\jan\.julia\v0.6\NullableArrays\src\operators.jl:99.
  likely near In[2]:1


,gene,cell,counts
1,A,1,302
2,B,1,4
3,C,1,350
4,D,1,2
5,E,1,35
6,F,1,0


In [4]:
by(df, [:gene], df -> mean(df[:counts]))

Base.AsyncCondition instead.
  likely near In[2]:1


,gene,x1
1,A,116.29
2,B,2.54
3,C,162.13
4,D,1.38
5,E,13.85
6,F,1.84
7,G,4.93
8,H,33.94
9,I,10.87
10,J,1.73


In [5]:
by(df, [:cell], df -> mean(df[:counts]))[:x1]

100-element DataArrays.DataArray{Float64,1}:
  82.4
   2.2
   5.2
   9.4
  15.5
  17.3
   5.1
  11.9
  10.9
 127.1
  71.1
  12.9
   8.9
   ⋮  
  23.7
   7.7
  19.1
  44.1
  77.4
  13.8
  48.8
  65.9
  17.9
 305.2
  19.4
  10.8

In [6]:
mm1 = fit!(glmm(@formula(counts ~ 1 + (1 | gene) +  (1 | cell)), df, Normal()))

Generalized Linear Mixed Model fit by minimizing the Laplace approximation to the deviance
  Formula: counts ~ 1 + (1 | gene) + (1 | cell)
  Distribution: Distributions.Normal{Float64}
  Link: GLM.IdentityLink()

  Deviance (Laplace approximation): 4670646.7538

Variance components:
          Column    Variance  Std.Dev. 
 cell (Intercept)  1928.7369 43.917387
 gene (Intercept)  2927.6297 54.107576

 Number of obs: 1000; levels of grouping factors: 100, 10

Fixed-effects parameters:
             Estimate Std.Error z value P(>|z|)
(Intercept)     34.95    17.665 1.97849  0.0479


In [17]:
mm2 = fit!(glmm(@formula(counts ~ 1 + (1 | gene) +  (1 | cell)), df, Poisson(), GLM.LogLink()))

in callback catch


LoadError: [91mBase.LinAlg.PosDefException(10)[39m

In [18]:
mm3 = fit!(glmm(@formula(counts ~ 1 + (1 | gene) +  (1 | cell)), df, Poisson(), GLM.LogLink()), fast=true)


in callback catch


LoadError: [91mBase.LinAlg.PosDefException(10)[39m

In [8]:
using DataFrames, MixedModels
p = 10;   # number of genes
n = 1000;  # number of cells
df = DataFrame(gene = repeat(1:p, outer=n), cell = repeat(1:n, inner=p), counts = zeros(n * p));
lmm1 = lmm(@formula(counts ~ 1 + (1|cell) + (1|gene)), df);
# set the random number generator seed for reproducibility
srand(1);
# simulate the linear mixed model response and fit it (as a sanity check)
σ=0.1;
refit!(simulate!(lmm1, β=[2.0], σ=σ, θ=[1., 2.]/σ));
# simulate the Poisson response
df[:counts] = rand.(Poisson.(exp.(model_response(lmm1))));
# try to fit poisson mixed-model
glmm1 = glmm(@formula(counts ~ 1 + (1|cell) + (1|gene)), df, Poisson())
fit!(glmm1, fast=true, verbose=true)


f_1: 26150.69115 [1.0, 1.0]
in callback catch


LoadError: [91mBase.LinAlg.PosDefException(5)[39m

In [9]:
glmm1

DomainError: DomainError:
log will only return a complex result if called with a complex argument. Try log(complex(x)).

In [94]:
glm1 = glm(@formula(counts ~ 1 + cell + gene), df, NegativeBinomial(), GLM.LogLink())
#fit!(glmm1, fast=true, verbose=true)
fit!(glm1, fast=true, verbose=true)


LoadError: [91mMethodError: no method matching mustart(::Distributions.NegativeBinomial{Float64}, ::Float64, ::Float64)[0m
Closest candidates are:
  mustart([91m::Distributions.Bernoulli[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:306
  mustart([91m::Distributions.Binomial[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:307
  mustart([91m::Union{Distributions.Gamma, Distributions.InverseGaussian}[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:308
  ...[39m

In [91]:
glmm1 = glmm(@formula(counts ~ 1 + (1|cell) + (1|gene)), df, NegativeBinomial(), GLM.LogLink())
fit!(glmm1, fast=true, verbose=true)

LoadError: [91mMethodError: no method matching mustart(::Distributions.NegativeBinomial{Float64}, ::Float64, ::Float64)[0m
Closest candidates are:
  mustart([91m::Distributions.Bernoulli[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:306
  mustart([91m::Distributions.Binomial[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:307
  mustart([91m::Union{Distributions.Gamma, Distributions.InverseGaussian}[39m, ::Any, ::Any) at C:\Users\jan\.julia\v0.6\GLM\src\glmtools.jl:308
  ...[39m

In [87]:
using DataFrames, MixedModels, RCall
p = 10;   # number of genes
n = 1000;  # number of cells
df = DataFrame(gene = repeat(string.(1:p), outer=n), cell = repeat(string.(1:n), inner=p), counts = zeros(n * p));
lmm1 = lmm(@formula(counts ~ 1 + (1|cell) + (1|gene)), df);
# set the random number generator seed for reproducibility
srand(1);
# simulate the linear mixed model response and fit it (as a sanity check)
σ=0.001;
refit!(simulate!(lmm1, β=[2.0], σ=σ, θ=[1.0, 2.0]/σ));
# simulate the Poisson response
df[:counts] = rand.(Poisson.(exp.(model_response(lmm1))));
# try to fit poisson mixed-model
pool!(df, [:cell, :gene])
glmm1 = fit!(glmm(@formula(counts ~ 1+ (1|cell)+ (1|gene)), df, Poisson()), fast=false, verbose=true)


f_1: 16662.90395 [1.0, 1.0]
in callback catch


LoadError: [91mBase.LinAlg.PosDefException(1)[39m

In [85]:
std(coeftable(glmm1).cols[1]), mean(coeftable(glmm1).cols[1])

(2.609660492852513, 3.2773577379274617)

In [79]:
@rput df
R"""
library(glmmTMB)
glmmTMB(counts~1+(1|gene)+(1|cell), data=df, family="poisson")
"""

RCall.RObject{RCall.VecSxp}
Formula:          counts ~ 1 + (1 | gene) + (1 | cell)
Data: df
      AIC       BIC    logLik  df.resid 
 58424.42  58446.05 -29209.21      9997 
Random-effects (co)variances:

Conditional model:
 Groups Name        Std.Dev.
 cell   (Intercept) 0.9935  
 gene   (Intercept) 2.6065  

Number of obs: 10000 / Conditional model: cell, 1000; gene, 10

Fixed Effects:

Conditional model:
(Intercept)  
      2.349  
